In [229]:
import sys
sys.path.append("../../")
sys.path.append("/home/ray__/CS/org/etherlabs/community_detection/")
import json as js
import text_preprocessing.preprocess as tp
import torch
import logging
import networkx as nx
import math
from scipy.spatial.distance import cosine
import community
from datetime import datetime
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert.modeling import BertPreTrainedModel, BertPreTrainingHeads
import numpy as np
import iso8601
bert_config = {}
bert_config["tokenizer"] = 'bert-base-uncased'
bert_config["config"] = '/home/ray__/CS/org/etherlabs/ai-engine_pants/ai-engine/mind_files/bert_config.json'
bert_config["bert_model"] = 'bert-base-uncased'
bert_config["load_file"] = "/home/ray__/CS/org/etherlabs/ai-engine_pants/ai-engine/mind_files/"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class BertForPreTraining_custom(BertPreTrainedModel):
    def __init__(self, config):
        super(BertForPreTraining_custom, self).__init__(config)
        self.bert = BertModel(config)
        self.cls = BertPreTrainingHeads(config, self.bert.embeddings.word_embeddings.weight)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, masked_lm_labels=None, next_sentence_label=None):
        output_all_encoded_layers=True
        sequence_output, pooled_output = self.bert(input_ids, token_type_ids, attention_mask,
                                                   output_all_encoded_layers=output_all_encoded_layers)
        if output_all_encoded_layers:
            sequence_output_pred = sequence_output[-1]
        prediction_scores, seq_relationship_score = self.cls(sequence_output_pred, pooled_output)

        return prediction_scores, seq_relationship_score, sequence_output, pooled_output 

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def getNSPScore(sample_text):
    m = torch.nn.Softmax()

    tokenized_text = tokenizer.tokenize(sample_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    segments_ids = [0]*tokenized_text.index('[SEP]')+[1]*(len(tokenized_text)-tokenized_text.index('[SEP]'))

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    pred_score, seq_rel, seq_out, pool_out = model1(tokens_tensor, segments_tensors)
    return m(seq_rel).detach().numpy()[0][0] #returns probability of being next sentence

def getSentMatchScore(sent1, sent2, nsp_dampening_factor = 0.7):
    sent1_feats = getBERTFeatures(model1, sent1, attn_head_idx)
    sent2_feats = getBERTFeatures(model1, sent2, attn_head_idx)
    cosine_distance = 1- cosine(sent1_feats, sent2_feats)
    nsp_input1 = sent1+' [SEP] '+sent2
    nsp_input2 = sent2+' [SEP] '+sent1
    nsp_score_1 = getNSPScore(nsp_input1)
    nsp_score_2 = getNSPScore(nsp_input2)
    nsp_score = np.mean([nsp_score_1,nsp_score_2])*nsp_dampening_factor
    len_diff = abs(len(sent1.split(' '))-len(sent2.split(' ')))
    if len_diff>2*(min(len(sent1.split(' ')),len(sent2.split(' ')))):
        #give more weight to nsp if the sentences of largely varying lengths
        score = 0.4*cosine_distance+0.6*nsp_score
    else:
        score = np.mean([cosine_distance,nsp_score])
    #print ("nsp score -> " + str(nsp_score))
    #print ("cosine score -> " + str(cosine_distance))
    return score

def getSentMatchScore_wfeature(sent1, sent2, sent1_feats, sent2_feats, nsp_dampening_factor = 0.7):
    cosine_distance = 1-cosine(sent1_feats, sent2_feats)
    #return cosine_distance
    nsp_input1 = sent1+' [SEP] '+sent2
    #nsp_input2 = sent2+' [SEP] '+sent1
    nsp_score_1 = getNSPScore(nsp_input1)
    #nsp_score_2 = getNSPScore(nsp_input2)
    nsp_score = nsp_score_1 * nsp_dampening_factor
    #nsp_score = nsp_score_1*nsp_dampening_factor
    len_diff = abs(len(sent1.split(' '))-len(sent2.split(' ')))
    if len_diff>2*(min(len(sent1.split(' ')),len(sent2.split(' ')))):
        #give more weight to nsp if the sentences of largely varying lengths
        score = 0.4*cosine_distance+0.6*nsp_score
    else:
        score = np.mean([cosine_distance,nsp_score])
    return score

def getSentMatchScore_wfeature_cosine(sent1, sent2, sent1_feats, sent2_feats, nsp_dampening_factor = 0.7):
    cosine_distance = 1-cosine(sent1_feats, sent2_feats)
    return cosine_distance

def getSentMatchScore_wfeature_test(sent1, sent2, sent1_feats, sent2_feats, nsp_dampening_factor = 0.7):
    cosine_distance = 1-cosine(sent1_feats, sent2_feats)
    nsp_input1 = sent1+' [SEP] '+sent2
    nsp_input2 = sent2+' [SEP] '+sent1
    nsp_score_1 = getNSPScore(nsp_input1)
    nsp_score_2 = getNSPScore(nsp_input2)
    nsp_score = np.mean([nsp_score_1,nsp_score_2])*nsp_dampening_factor
    #nsp_score = nsp_score_1*nsp_dampening_factor
    len_diff = abs(len(sent1.split(' '))-len(sent2.split(' ')))
    if len_diff>2*(min(len(sent1.split(' ')),len(sent2.split(' ')))):
        #give more weight to nsp if the sentences of largely varying lengths
        score = 0.4*cosine_distance+0.6*nsp_score
    else:
        score = np.mean([cosine_distance,nsp_score])
    return score, cosine_distance, nsp_score

def getBERTFeatures(model, text, attn_head_idx = -1): #attn_head_idx - index o[]
    tokenized_text = tokenizer.tokenize(text)
    if len(tokenized_text)>200:
        tokenized_text = tokenized_text[0:200]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    _, _, seq_out, pool_out = model(tokens_tensor)
    seq_out = list(getPooledFeatures(seq_out[attn_head_idx]).T)
    #pool_out = list(pool_out.detach().numpy().T)
    return seq_out

def getPooledFeatures(np_array):
    np_array = np_array.reshape(np_array.shape[1],np_array.shape[2]).detach().numpy()
    np_array_mp = np.mean(np_array, axis=0).reshape(1, -1)
    return np_array_mp

def replaceContractions(text):
    #text = text.lower()
    c_filt_text = ''
    for word in text.split(' '):
        if word in contractions:
            c_filt_text = c_filt_text+' '+contractions[word]
        else:
            c_filt_text = c_filt_text+' '+word
    return c_filt_text

def cleanText(text):
    text = text.replace('\\n','')
    text = text.replace('\\','')
    #text = text.replace('\t', '')
    #text = re.sub('\[(.*?)\]','',text) #removes [this one]
    text = re.sub('(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?\s',
                ' __url__ ',text) #remove urls
    #text = re.sub('\'','',text)
    #text = re.sub(r'\d+', ' __number__ ', text) #replaces numbers
    text = re.sub('\W', ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    return text

def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

import json
from dataclasses import dataclass, asdict
from typing import List
import logging
import text_preprocessing.preprocess as tp


logger = logging.getLogger()


@dataclass
class Request:
    segments: list

def decode_json_request(body) -> Request:
    req = body

#     if isinstance(body, str):
#         req = json.load(body)

    def decode_segments(seg):
        segments_text = list(map(lambda x:tp.preprocess(x['originalText'], stop_words=False, remove_punct=False), seg['segments']))
        segments_data = seg['segments']
        for index, segment in  enumerate(segments_data):
            segments_data[index]['originalText'] = segments_text[index]
        return segments_data

    segments = decode_segments(req)
    return Request(segments)

from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert.modeling import BertPreTrainedModel, BertPreTrainingHeads
import logging

logger = logging.getLogger()

def loadmodel(model_config, mind):
    tokenizer = BertTokenizer.from_pretrained(model_config['tokenizer'])
    config_file = BertConfig.from_json_file(model_config['config'])
    #logger.debug("config file", extra={"contains": model_config})
    bert_model = model_config['bert_model']
    load_file = model_config['load_file'] + mind
# config_file = BertConfig.from_json_file('services/community/bert_config.json')
    model1 = BertForPreTraining_custom(config_file)
    state_dict_1 = torch.load(load_file, map_location='cpu')
    model1.load_state_dict(state_dict_1)
    return model1

def selectmodel(MindId):
    return "mind-"+str(MindId)+".bin"


In [230]:
logger = logging.getLogger()
class community_detection():
    segments_list = []
    model1 = None
    def __init__(self, Request, model1):
        self.segments_list = Request.segments
        self.model1 = model1
    #def parse_meeting(self, segments):
    #    segments_data = list(map(lambda x: tp.preprocess(x['originalText'], stop_words=False, remove_punct=False), segments['segments']))
    #    self.segments_list = segments['segments']
    #    for index,seg in enumerate(self.segments_list):
    #        self.segments_list[index]['originalText'] = segments_data[index]

    def compute_feature_vector(self):
        #tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #config = BertConfig.from_json_file('services/community/bert_config.json')
        #bert_model = 'bert-base-uncased'
        #model1 = bert.BertForPreTraining_custom(config)
        #state_dict_1 = torch.load('services/community/bert_10epc_se_1e-6_sl40.bin')
        #state_dict_1 = torch.load('services/community/bert_10epc_inc_se+etherdata_1e-6_sl40_bt64.bin')
        #model1.load_state_dict(state_dict_1)
        #model1.eval()
        graph_list = {}
        fv = {}
        index = 0
        for segment in self.segments_list:
            for sent in segment['originalText']:
                graph_list[index] = (sent, segment['startTime'], segment['spokenBy'], segment['id'])
                fv[index] = getBERTFeatures(self.model1, sent, attn_head_idx=-1)
                index+=1
        return fv, graph_list

    def construct_graph(self, fv, graph_list):
        meeting_graph = nx.Graph()
        yetto_prune = []
        c_weight = 0
        for indexa, nodea in enumerate(graph_list.values()):
            for indexb, nodeb in enumerate(graph_list.values()):
                if indexb>indexa:
                    c_weight = cosine(fv[indexa], fv[indexb])
                    meeting_graph.add_edge(indexa, indexb, weight=c_weight)
                    yetto_prune.append((indexa, indexb, c_weight))
        return meeting_graph, yetto_prune

    def prune_edges(self, meeting_graph, graph_list, yetto_prune,v=0.01):
        yetto_prune = sorted(yetto_prune, key=lambda kv : kv[2], reverse=True)
        yetto_prune = yetto_prune[:math.ceil(len(yetto_prune)*v)+1]
        #logger.info("pruning value", extra={"v is : ": v})
        print ({"v is : ": v})
        meeting_graph_pruned = nx.Graph()
        for indexa, indexb, c_score in yetto_prune:
            meeting_graph_pruned.add_edge(indexa, indexb)
        return meeting_graph_pruned

    def compute_louvian_community(self, meeting_graph_pruned, community_set):
        #community_set = community.best_partition(meeting_graph_pruned,randomize=False,random_state=9)
        #modularity_score = community.modularity(community_set, meeting_graph_pruned)
        #logger.info("Community results", extra={"modularity score":modularity_score})
        #print ({"modularity score":modularity_score})
        community_set_sorted = sorted(community_set.items(), key = lambda kv: kv[1], reverse=False)

        return community_set_sorted

    def refine_community(self, community_set_sorted, graph_list):
        clusters = []
        temp = []
        prev_com = 0
        for word,cluster in community_set_sorted:
            if prev_com==cluster:
                temp.append(word)
            else:
                clusters.append(temp)
                temp = []
                prev_com = cluster
                temp.append(word)
        timerange = []
        temp = []
        for cluster in clusters:
            temp= []
            for sent in cluster:
                #temp.append(graph_list[sent])
                #logger.info("segment values", extra={"segment":self.segments_list})
                temp.append(graph_list[sent])
            if len(temp)!=0:
                temp = list(set(temp))
                temp = sorted(temp,key=lambda kv: kv[1], reverse=False)
                timerange.append(temp)

        return timerange

    def group_community_by_time(self, timerange):
        timerange_detailed = []
        temp = []
        flag = False
        pims = {}
        index_pim = 0
        index_segment = 0
        for index,com in enumerate(timerange):
            temp = []
            flag = False
            #print ("-----community-----", index)
            for (index1,(sent1,time1,user1, id1)), (index2,(sent2,time2,user2, id2)) in zip(enumerate(com[0:]),enumerate(com[1:])):
                if id1!=id2:
                    if ((formatTime( time2, True)-formatTime(time1, True)).seconds<=240):
                        if (not flag):
                            pims[index_pim] = {'segment'+str(index_segment):[sent1,time1,user1, id1]}
                            index_segment+=1
                            temp.append((sent1,time1,user1, id1))
                        pims[index_pim]['segment'+str(index_segment)] = [sent2,time2,user2, id2]
                        index_segment+=1
                        temp.append((sent2,time2,user2,id2))
                        flag=True
                    else:
                        if flag==True:
                            index_pim+=1
                            index_segment=0
                        flag=False
            if flag==True:
                index_pim+=1
                index_segment=0
            timerange_detailed.append(temp)
        return pims

    def wrap_community_by_time(self, pims):
        yet_to_combine = []
        need_to_remove = []
        inverse_dangling_pims = []
        for index1,i in enumerate(pims.keys()):
            for index2,j in enumerate(pims.keys()):
                if index1!=index2:
                    if pims[i]['segment0'][1] >= pims[j]['segment0'][1] and pims[i]['segment0'][1] <= pims[j]['segment'+str(len(pims[j].values())-1)][1]:
                        if (j,i) not in yet_to_combine and i not in need_to_remove and j not in need_to_remove:
                            yet_to_combine.append((i,j))
                            need_to_remove.append(i)
        for i,j in yet_to_combine:
            for k in pims[i]:
                if pims[i][k] not in pims[j].values():
                    pims[j]['segment'+str(len(pims[j].values())-1)] = pims[i][k]
                    continue
        for i in need_to_remove:
            pims.pop(i)

        for index, p in enumerate(pims.keys()):
            for seg in pims[p].keys():
                pims[p][seg][0] = [' '.join(text for text in segment['originalText']) for segment in self.segments_list if segment['id']==pims[p][seg][3]]
                inverse_dangling_pims.append(pims[p][seg][3])

        c_len = 0
        for segment in self.segments_list:
            if segment['id'] not in inverse_dangling_pims:
                while c_len in pims.keys():
                    c_len+=1
                pims[c_len] = {"segment0": [' '.join(text for text in segment['originalText']), segment['startTime'], segment['spokenBy'], segment['id']]}
        return pims

    def get_communities(self):
        segments_data = ' '.join([sentence for segment in self.segments_list for sentence in segment['originalText']])
        fv, graph_list = self.compute_feature_vector()
        logger.info("No of sentences is", extra={"sentence": len(fv.keys())})
        meeting_graph, yetto_prune = self.construct_graph(fv, graph_list)
        for v in [0.15, 0.1, 0.05, 0.01]:
            flag = False
            for i in range(5):
                meeting_graph_pruned =  self.prune_edges(meeting_graph, graph_list, yetto_prune, v)
                #community_set = community.best_partition(meeting_graph_pruned,randomize=False,random_state=9)
                community_set = community.best_partition(meeting_graph_pruned)
                mod = community.modularity(community_set, meeting_graph_pruned)
                #logger.info("Meeting Graph results", extra={"edges before prunning":meeting_graph.number_of_edges(), "edges after prunning": meeting_graph_pruned.number_of_edges()})
                print ({"edges before prunning":meeting_graph.number_of_edges()}, {"edges after prunning": meeting_graph_pruned.number_of_edges()},{"Modularity": mod})
                if mod>0.3:
                    print ("got modularity greater than 3. breaking.")
                    flag = True
                    break
                elif mod==0:
                    meeting_graph_pruned = self.prune_edges(meeting_graph, graph_list, yetto_prune, 0.15)
                    flag = True
                    break
            if flag:
                break
        #print({"edges before prunning":meeting_graph.number_of_edges()}, {"edges after prunning": meeting_graph_pruned.number_of_edges()})
        community_set_sorted = self.compute_louvian_community(meeting_graph_pruned, community_set)
        #print (community_set_sorted)
        community_timerange = self.refine_community(community_set_sorted, graph_list)
       #logger.info("commnity timerange", extra={"timerange": community_timerange})
        pims = self.group_community_by_time(community_timerange)
        pims = self.wrap_community_by_time(pims)

        logger.info("Final PIMs", extra={"PIMs": pims})
        return pims


In [231]:
def computepims(segments, model1):
    community_extraction = community_detection(segments, model1)
    pims = community_extraction.get_communities()
    return pims

In [232]:
with open("curl_test.json", 'r') as f:
    requests = json.load(f)

In [233]:
segments = decode_json_request(requests)
mind = selectmodel("01daayheky5f4e02qvrjptftxv")
model1 = loadmodel(bert_config, mind)
#print (requests['segments'])
res = computepims(segments, model1)

{'v is : ': 0.15}
{'edges before prunning': 11628} {'edges after prunning': 1746} {'Modularity': 0.20707698434254562}
{'v is : ': 0.15}
{'edges before prunning': 11628} {'edges after prunning': 1746} {'Modularity': 0.2053597553694978}
{'v is : ': 0.15}
{'edges before prunning': 11628} {'edges after prunning': 1746} {'Modularity': 0.20707698434254562}
{'v is : ': 0.15}
{'edges before prunning': 11628} {'edges after prunning': 1746} {'Modularity': 0.19635931056291}
{'v is : ': 0.15}
{'edges before prunning': 11628} {'edges after prunning': 1746} {'Modularity': 0.2053597553694978}
{'v is : ': 0.1}
{'edges before prunning': 11628} {'edges after prunning': 1164} {'Modularity': 0.23260936632774767}
{'v is : ': 0.1}
{'edges before prunning': 11628} {'edges after prunning': 1164} {'Modularity': 0.24647168491160942}
{'v is : ': 0.1}
{'edges before prunning': 11628} {'edges after prunning': 1164} {'Modularity': 0.22179377605366019}
{'v is : ': 0.1}
{'edges before prunning': 11628} {'edges after 

In [234]:
res

{0: {'segment0': [['All the changes accomplished complete on Subscription on two things I noticed test was that I IP address in the main link. So for that we I will discuss and and.'],
   '2019-07-01T05:51:47Z',
   '5aa6f27a-c06e-4569-9117-5797fafe8356',
   '3a83f8cd4b014a62a3160e8cc318bff3'],
  'segment1': [['Very and so that that will not happen and will that for the we need the then function that website video only allen.'],
   '2019-07-01T05:52:05Z',
   '5aa6f27a-c06e-4569-9117-5797fafe8356',
   '56ae9ca08ba141fd83e39984660f9502'],
  'segment2': [['Admin page right now it is just the basic so what the Colin had pointed out was that if we can make it look like the ether dashboard page in the background and and all that yeah.'],
   '2019-07-01T05:52:39Z',
   '5aa6f27a-c06e-4569-9117-5797fafe8356',
   '9144799661c34afb9bd7ed9cfddd95cd'],
  'segment3': [['Should I just remove that bton from website and in in the moment if the it is better.'],
   '2019-07-01T05:54:35Z',
   '5aa6f27a-c06